# Work plan
1. Download data
1. Organize data into folders
1. Finetune vgg model
1. Train model
1. Run predictions
1. Evaluate predictions
1. Submit to Kaggle

## Download data
Done offline with Kaggle command line utility in Terminal ssh session. Unzip files.

Confirmed below

[Download_link](https://www.kaggle.com/c/invasive-species-monitoring/data)

## Organize data into folders
only run once

In [2]:
# frequent file shortcuts
import os, sys
current_directory = os.getcwd()
UTIL_DIR = '/home/laudai/AnacondaProjects/laudai_DLtest/utils'
LESSON_HOME_DIR = current_directory
DATA_HOME_DIR = current_directory+'/data/invasive_species'

In [3]:
# set path variables
sys.path.insert(1, LESSON_HOME_DIR)
sys.path.insert(1, UTIL_DIR)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [4]:
sys.path

['',
 '..',
 '/home/laudai/AnacondaProjects/laudai_DLtest/utils',
 '/home/laudai/AnacondaProjects/laudai_DLtest',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python27.zip',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/plat-linux2',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/lib-tk',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/lib-old',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/lib-dynload',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages/PIL',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages/IPython/extensions',
 '/home/laudai/.ipython']

In [5]:

#import utility modules
from utils import *
#plotting in jupyter notebook
%matplotlib inline

Using TensorFlow backend.


In [6]:
!pip list | grep Keras
#laudai list Keras version

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
Keras (1.2.2)


In [31]:
# make directories
#make your directories have data dir
"""


%mkdir -p data/invasive_species/train/invasive
%mkdir -p data/invasive_species/train/harmless
%mkdir -p data/invasive_species/valid/invasive
%mkdir -p data/invasive_species/valid/harmless
%mkdir -p data/invasive_species/test/unknown
%mkdir -p data/invasive_species/sample/train/invasive
%mkdir -p data/invasive_species/sample/train/harmless
%mkdir -p data/invasive_species/sample/valid/invasive
%mkdir -p data/invasive_species/sample/valid/harmless
%mkdir -p data/invasive_species/results

"""

In [8]:
#confirm training data & csv file with labels correctly loaded
#downloaded and unpacked in terminal. 

%cd $DATA_HOME_DIR/train
%ls *.jpg | head -5



print("\ntrain_labels.csv")
!head -5 train_labels.csv

/home/laudai/AnacondaProjects/laudai_DLtest/data/invasive_species/train
ls: 無法存取 '*.jpg': 沒有此一檔案或目錄

train_labels.csv
name,invasive
1,0
2,0
3,1
4,0


In [9]:
#need to move images with '1' label into invasive folder and those with '0' to harmless folder
import csv
import numpy as np

reader = csv.reader(open("train_labels.csv", "rb"), delimiter=",")
labels = list(reader)
del labels[0]
labels[0:5]

[['1', '0'], ['2', '0'], ['3', '1'], ['4', '0'], ['5', '1']]

In [10]:
#confirm labels for all training data. 2295 from contest page.
len(labels) == 2295

True

In [ ]:
#if you run first , u should run this block
"""
for l in labels:
    iname = l[0] + ".jpg"
    if l[1] == '0':
        !mv -t harmless/ $iname
    elif l[1]== '1':
        !mv -t invasive/ $iname

"""
#run once to move image stroge



In [11]:
ls

harmless/  invasive/  train_labels.csv


In [12]:
%cd $DATA_HOME_DIR
!tree -d


#!pwd
#laudai
#if you don't have use apt-get install tree

/home/laudai/AnacondaProjects/laudai_DLtest/data/invasive_species
.
├── results
│   ├── 54231_filenames.dat
│   │   ├── data
│   │   └── meta
│   ├── 54231_preds.dat
│   │   ├── data
│   │   └── meta
│   ├── 54242_filenames.dat
│   │   ├── data
│   │   └── meta
│   └── 54242_preds.dat
│       ├── data
│       └── meta
├── sample
│   ├── train
│   │   ├── harmless
│   │   └── invasive
│   └── valid
│       ├── harmless
│       └── invasive
├── test
│   └── unknown
├── train
│   ├── harmless
│   └── invasive
└── valid
    ├── harmless
    └── invasive

28 directories


In [13]:
!pwd
#%cd $DATA_HOME_DIR
!ls train/
!ls train/harmless | wc -l
!ls train/invasive/ | wc -l
# to dispaly file count

/home/laudai/AnacondaProjects/laudai_DLtest/data/invasive_species
harmless  invasive  train_labels.csv
763
1057


## Finetune vgg model
using vgg16.py file from fast.ai course. Model pre-trained on Imagenet. 

See https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16.py

In [14]:
# more file shortcuts

#Set path to /sample for development speed
# path = DATA_HOME_DIR + '/sample'


"""
DATA_HOME_DIR = "/home/laudai/AnacondaProjects/laudai_DLtest/data/invasive_species"
"""



path = DATA_HOME_DIR
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

In [15]:
print train_path
os.listdir(train_path)

/home/laudai/AnacondaProjects/laudai_DLtest/data/invasive_species/train/


['train_labels.csv', 'invasive', 'harmless']

In [16]:
#instantiate pretrained model
from vgg16 import Vgg16
vgg = Vgg16()

In [17]:
#finetune model with our data
batches = vgg.get_batches(train_path)
#laudai


# In this you will get problem , shutdown this project right now to toggle another project
print batches
vgg.finetune(batches)



Found 1820 images belonging to 2 classes.


## Train model

#### 解決無法使用 l1 l2 activity_l1 activity_l2的方法
#### 就是到github 上去找有沒有哪個版本是有這個的方法，再去灌這個版本 最後使用keras1.2.2
[github 版本位置連結](https://github.com/fchollet/keras/blob/1.2.2/keras/regularizers.py)

In [21]:
#set attributes
num_epochs = 4
lr = 0.001
import time

In [19]:
train_batches = vgg.get_batches(train_path)
valid_batches = vgg.get_batches(valid_path)
ts = str(int(time.time()))[-5:]
latest_weights_filename = None

for epoch in range(num_epochs):
    print("\nrunning epoch %d" % epoch)
    vgg.fit(train_batches, valid_batches)
    latest_weights_filename = ts + '_vgg_%d.h5' % epoch
    vgg.model.save_weights(results_path + latest_weights_filename)

Found 1820 images belonging to 2 classes.
Found 475 images belonging to 2 classes.

running epoch 0
Epoch 1/1
  40/1820 [..............................] - ETA: 4029s - loss: 0.9443 - acc: 0.6500

KeyboardInterrupt: 

## Evaluate model
Evaluate some images to check our results

In [ ]:
#latest_weights_filename ='77272_vgg_4.h5'
latest_weights_filename ='69904_vgg_3.h5'
#use your vgg.h5 (PATH in RESULT dir)

In [ ]:
#load weights and generate predictions of validation set for review
vgg.model.load_weights(results_path + latest_weights_filename)
valid_batches, valid_preds = vgg.test(valid_path)
valid_filenames = valid_batches.filenames
expected_labels = valid_batches.classes

In [ ]:
valid_batches.class_indices

In [ ]:
#round off predictions to 0 or 1 to make labels
s = valid_preds[:5]
s

In [ ]:
#second column is probability of invasive
s[:,1]

In [ ]:
our_predictions = valid_preds[:,1]
#our_string_predictions = np.asarray(["%.2f" % p for p in our_predictions], dtype='|S9') #for titles- not working
our_labels = np.round(our_predictions)

In [ ]:
from keras.preprocessing import image

#Helper function to plot images by index in the validation set 
#Plots is a helper function in utils.py
def plots_idx(idx, titles):
    plots([image.load_img(valid_path + valid_filenames[i]) for i in idx], titles=titles)
    
#Number of images to view for each visualization task
n_view = 4

### Correctly labeled images

In [ ]:
#selected at random
correct = np.where(our_labels == expected_labels)[0]
print("found %d correct labels" % len(correct))
idx = np.random.permutation(correct)
titles = np.where(our_labels[idx[:n_view]] == 0.0, 'harmless', 'invasive')
plots_idx(idx[:n_view], titles)

In [ ]:
idx = np.random.permutation(correct)
titles = np.where(our_labels[idx[:n_view]] == 0.0, 'harmless', 'invasive')
plots_idx(idx[:n_view], titles)

In [ ]:
idx = np.random.permutation(correct)
titles = np.where(our_labels[idx[:n_view]] == 0.0, 'harmless', 'invasive')
plots_idx(idx[:n_view], titles)

### Incorrectly labeled images

In [ ]:
#selected at random
incorrect = np.where(our_labels != expected_labels)[0]
print("found %d incorrect labels" % len(incorrect))
idx = np.random.permutation(incorrect)
titles = np.where(our_labels[idx[:n_view]] == 0.0, 'harmless:', 'invasive')
plots_idx(idx[:n_view], titles)

In [ ]:
idx = np.random.permutation(incorrect)
titles = np.where(our_labels[idx[:n_view]] == 0.0, 'harmless:', 'invasive')
plots_idx(idx[:n_view], titles)

### Images with highest uncertainty

In [ ]:
most_uncertain = np.argsort(abs(our_predictions-0.5))
plots_idx(most_uncertain[:n_view], our_predictions[most_uncertain[:n_view]])

### Confustion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)
plot_confusion_matrix(cm, valid_batches.class_indices)

## Run predictions

In [17]:
#preds is a numpy array of predictions


test_batches, preds = vgg.test(test_path)

Found 0 images belonging to 1 classes.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages/keras/engine/training.py", line 429, in data_generator_task
    generator_output = next(self._generator)
  File "/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages/keras/preprocessing/image.py", line 822, in next
    index_array, current_index, current_batch_size = next(self.index_generator)
  File "/home/laudai/anaconda2/envs/python27_DL/lib/python2.7/site-packages/keras/preprocessing/image.py", line 645, in _flow_index
    current_index = (self.batch_index * batch_size) % n
ZeroDivisionError: integer division or modulo by zero



In [42]:
#save predictions
filenames = test_batches.filenames
ts = str(int(time.time()))[-5:]
save_array(results_path + ts + '_preds.dat', preds)
save_array(results_path + ts + '_filenames.dat', filenames)

## Evaluate predictions

In [43]:
from PIL import Image
evals = 10
image_display_size = 512, 512

In [44]:
preds[:evals]

[]

In [45]:
testfile = 0
print("testfile %d" % testfile + np.array_str(preds[testfile]))
im = Image.open(test_path + filenames[testfile])
im.resize(image_display_size)

IndexError: list index out of range

## Submit to Kaggle
Create file then submit offline

In [274]:
%cd $results_path
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
is_invasive = preds[:,1]
submission = np.stack((ids, is_invasive), axis = 1)
filename = "kevin" + ts + ".csv"
np.savetxt(filename, submission, fmt="%d,%.5f", header='name,invasive', comments='')

/home/ubuntu/nbs/lesson1/data/invasive_species/results
